[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/salesforce/GeDi/blob/master/GeDi_guided_GPT_2_XL.ipynb)

Official implementation of generation with the topic GeDi (pronounced *Jedi*) model based on our paper [GeDi: Generative Discriminator Guided Sequence Generation](https://arxiv.org/abs/2009.06367)

Check our github repository for more options (like detoxification and sentiment control) https://github.com/salesforce/GeDi

In [ ]:
!wget https://storage.googleapis.com/sfr-gedi-data/GeDi.zip
import zipfile
with zipfile.ZipFile('GeDi.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

--2021-01-23 10:32:05--  https://storage.googleapis.com/sfr-gedi-data/GeDi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.217.13.240, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 260070 (254K) [application/zip]
Saving to: ‘GeDi.zip’

GeDi.zip            100%[===================>] 253.97K  --.-KB/s    in 0.002s  

2021-01-23 10:32:05 (114 MB/s) - ‘GeDi.zip’ saved [260070/260070]



In [ ]:
cd GeDi

/content/GeDi


In [ ]:
'''Installing transformers v2.8'''

!pip install transformers==2.8
!pip install -r hf_requirements.txt

'''Downloading GeDi topic model checkpoints'''

!wget https://storage.googleapis.com/sfr-gedi-data/gedi_topic.zip

with zipfile.ZipFile('gedi_topic.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

     |████████████████████████████████| 573kB 16.7MB/s 
     |████████████████████████████████| 133kB 58.4MB/s 
     |████████████████████████████████| 1.2MB 39.4MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
     |████████████████████████████████| 3.7MB 52.2MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 7.2MB 48.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=09042de99e89118a0619ac53b0d1b13e440ae22e24aa164b348cb077de0f3ceb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.59 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 317kB 14.4MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-no

In [ ]:
import numpy as np
import torch


from modeling_gpt2 import GPT2LMHeadModel

from transformers import (
    GPT2Config,
    GPT2Tokenizer
)

In [ ]:
mode = "topic"
code_desired = "true"
code_undesired = "false"
model_type = 'gpt2'
gen_type = "gedi"
gen_model_name_or_path = "gpt2-xl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),}
config_class, model_class, tokenizer_class = MODEL_CLASSES["gpt2"]
tokenizer = tokenizer_class.from_pretrained(gen_model_name_or_path, do_lower_case=False)



The next step needs to download and convert the GPT2-XL model. 

This takes a while (usually about 3 minutes to download and another 5 or so to convert). 

The good news is that once the model is loaded, you can quickly sample from many different prompts and topics.

In [ ]:
#Loading GPT2-XL model (1.5B param LM) below, this could take a while.
#This requires additional CPU memory overhead to load the pretrained weights in a new model
#Due to CPU memory constraints on Colab, we're loading the model in half precision (load_in_half_prec=True) 
#Do to this change, generations may not always exactly match samples in paper, but sometimes do, and seem to be similar in quality
#If you run the notebook with enough CPU RAM (most likely 16GB+), you can try setting load_in_half_prec=False   

model = model_class.from_pretrained(gen_model_name_or_path, load_in_half_prec=True)
model = model.to(device)
model = model.float()

gedi_model_name_or_path = 'gedi_topic'
gedi_model = model_class.from_pretrained(gedi_model_name_or_path)
gedi_model.to(device)


no logit scale initialized for gpt2


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): SharedDropout()
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): SharedDropout()
          (resid_dropout): SharedDropout()
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): SharedDropout()
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): SharedDropout()
          (resid_dropout): SharedDropout()
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
        

### Set arguments for generation

You can change the max generation length, or play around with hyperparameter settings. 

The default hyperparameters were used in the topic model for the paper.

More aggressive topic steering can be done by increasing `disc_weight` or `filter_p` (`filter_p` should always be less than 1)

In [ ]:
#setting arguments for generation
#max generation length
gen_length = 200
#omega from paper, higher disc_weight means more aggressive topic steering
disc_weight = 30
#1 - rho from paper, should be between 0 and 1 higher filter_p means more aggressive topic steering
filter_p = 0.8
#tau from paper, preserves tokens that are classified as correct topic
target_p = 0.8
#hyperparameter that determines class prior, set to uniform by default
class_bias = 0

if gen_length>1024:
  length = 1024
else:
  length = gen_length

### Specify prompt and topic to GeDi


The topic and prompt can be specified as strings with the `secondary_code` and `prompt` variables below.

Note that our GeDi topic model has been trained on only four topics:  `world`, `sports`, `business` and `science` so it performs best on steering generation from GPT-2 towards these topics. However, it also shows some promising zero-shot results on new topics for eg. `education`, `food`, `fire`, `space`, `cars`, `climate`.

Generic short prompts tend to work the best.

In [ ]:
#Specify what topic you want to generate on using the secondary_code variable

secondary_code = 'death'
bpe_tokens = tokenizer.encode(secondary_code)
if len(bpe_tokens) > 1:
  print("Warning! number of bpe tokens for " + code + " is greater than 1, model isn't trained for this, generation is less likely to match the topic")

In [ ]:
#Specify prompt below
prompt = """he quarantine started at 04.45 local time (21:45 CET). People living in about 200 apartment buildings in the Jordan district are forbidden to leave their apartments until they have all been tested for the presence of the SARS-CoV-2 coronavirus.
Around 3,000 civil servants are involved in organizing the testing, including more than 1,700 police, firefighters and immigration officials, according to the South China Morning Post.
Testing takes place at 51 mobile sites. The local government provided the population with basic food supplies during quarantine, writes AFP.
The authorities took the measure after 162 confirmed cases of infection in 56 apartment buildings were detected in the Jordan district between 1 and 20 January."""
start_len=0
text_ids = tokenizer.encode(prompt)
encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0).to(device)

multi_code = tokenizer.encode(secondary_code)
attr_class = 1

generated_sequence = model.generate(input_ids=encoded_prompts,
                                         pad_lens=None,
                                          max_length= length,
                                          top_k=None,
                                          top_p=None,
                                          repetition_penalty= 1.2,
                                          rep_penalty_scale= 10,
                                          eos_token_ids = tokenizer.eos_token_id,
                                          pad_token_id = 0,
                                          do_sample= False,
                                          penalize_cond= True,
                                          gedi_model= gedi_model,
                                          tokenizer= tokenizer,
                                          disc_weight= disc_weight,
                                          filter_p = filter_p,
                                          target_p = target_p,
                                          class_bias = class_bias,
                                          attr_class = attr_class,
                                          code_0 = code_undesired,
                                          code_1 = code_desired,
                                          multi_code=multi_code
                                          )

text = tokenizer.decode(generated_sequence.tolist()[0], clean_up_tokenization_spaces=True)
print('\n')
print(text)

GeDi estimates the probability that it sample is desired class is: 0.4988851845264435


he quarantine started at 04.45 local time (21:45 CET). People living in about 200 apartment buildings in the Jordan district are forbidden to leave their apartments until they have all been tested for the presence of the SARS-CoV-2 coronavirus.
Around 3,000 civil servants are involved in organizing the testing, including more than 1,700 police, firefighters and immigration officials, according to the South China Morning Post.
Testing takes place at 51 mobile sites. The local government provided the population with basic food supplies during quarantine, writes AFP.
The authorities took the measure after 162 confirmed cases of infection in 56 apartment buildings were detected in the Jordan district between 1 and 20 January. There were also seven deaths reported from people who came down with respiratory infections that did not match those found on tests conducted so far by Chinese authorities or those